# Identifying Hummingbird Species of the Rocky Mountains

*Adapted from FastAI v4, Course 2*


Audubon [observes the following](https://rockies.audubon.org/blog/habitat-hero/backyard-hummingbirds
) about hummingbird species in the Rockies:

> In Wyoming and Colorado there are four main species that you will see. *Broad-tailed* (Selasphorus platycercus) and *Rufous* (Selasphorous rufus) Hummingbirds are the most common you will probably see. Less common species are the *Calliope* (Selasphorus calliope) and *Black-chinned* (Archilochus alexandri) Hummingbirds.

|  |  |  |  |
|---|---|---|---|
| ![]( https://nas-national-prod.s3.amazonaws.com/styles/bird_illustration/s3/2735_Sibl_9780307957900_art_r1.jpg?itok=m1TzaJDi ) | ![]( https://nas-national-prod.s3.amazonaws.com/styles/bird_illustration/s3/2747_Sibl_9780307957900_art_r1.jpg?itok=LXCr5qsa ) | ![]( https://nas-national-prod.s3.amazonaws.com/styles/bird_illustration/s3/2760_Sibl_9780307957900_art_r1.jpg?itok=pnOj0j9q ) | ![]( https://nas-national-prod.s3.amazonaws.com/styles/bird_illustration/s3/2733_Sibl_9780307957900_art_r1.jpg?itok=kM3iVfa8 ) |


In [ ]:
#hide
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', 'WXYZ')

In [ ]:
search_images_bing

In [ ]:
results = search_images_bing(key, 'rufous hummingbird')
ims = results.attrgot('content_url')
len(ims)

In [ ]:
#hide
ims = ['https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Selasphorus_rufus_on_Saltspring_Island.jpg/1024px-Selasphorus_rufus_on_Saltspring_Island.jpg']

In [ ]:
dest = 'images/rufous.jpg'
download_url(ims[0], dest)

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
hummingbird_types = 'broad-tailed','rufous','calliope','black-chinned'
path = Path('hummingbirds')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in hummingbird_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} hummingbird')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
hummingbirds = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = hummingbirds.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
hummingbirds = hummingbirds.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = hummingbirds.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
hummingbirds = hummingbirds.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = hummingbirds.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
hummingbirds = hummingbirds.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = hummingbirds.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

### Data Augmentation

In [ ]:
hummingbirds = hummingbirds.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = hummingbirds.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
hummingbirds = hummingbirds.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = hummingbirds.dataloaders(path, bs=16)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

## Turning Your Model into an Online Application

### Using the Model for Inference

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('images/rufous.jpg')

In [ ]:
learn_inf.dls.vocab

### Creating a Notebook App from the Model

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
#hide
# For the book, we can't actually click an upload button, so we fake it
btn_upload = SimpleNamespace(data = ['images/rufous.jpg'])

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your Colorado hummingbird! (broad-tailed, rufous, calliope, black-chinned)'), 
      btn_upload, btn_run, out_pl, lbl_pred])